# This is Notebook is an introduction the the Koverse Python SDK

## Start by connecting to Koverse throught the client library

In [2]:
from koverse import client
client.connect()

## There are multiple ways to retreive data from Koverse 
* Spark Data Frame
* Pandas Data Frame
* Using SQL
* Query Koverse


In [2]:
import pandas as pd
import pprint

## Available Data Sets

In [3]:
pDatasets = pd.DataFrame(list([ {'name':x.name,'id':x.id,'recordCount':x.recordCount} for x in client.listDatasets()]))
pDatasets[0:10]

,name,id,recordCount
0,Synthea Imaging Studies,synthea_imaging_studies_20200518_224502_974,0
1,Synthea Immunizations,synthea_immunizations_20200518_224812_002,0
2,Titanic,titanic_20200511_190511_437,0
3,employees,employees_20200415_141342_249,0
4,TitanicSparkML,titanicsparkml_20200514_145006_279,0
5,Real Estate Sales 2001Thru2017,real_estate_sales_2001thru2017_20200609_204146...,0
6,Synthea Organizations,synthea_organizations_20200518_204638_543,0
7,Synthea Encounters,synthea_encounters_20200514_020629_175,0
8,LexisNexis Sample,lexisnexis_sample_20190815_025249_062,0
9,CME Crosswalk Test No Dates,cme_crosswalk_test_no_dates_20200113_214056_920,0


## First we will use a simple get spark dataframe

In [4]:
df = client.get_spark_dataframe(spark,data_set_name='employees')
df.show(5)

+------------+-------------------+--------------+-----------+----------------------+-------+----------+
|businessUnit|           hireDate|          name|orgCategory|orgCategoryDescription|orgCode|  traderId|
+------------+-------------------+--------------+-----------+----------------------+-------+----------+
|          PB|2010-01-12 17:46:23| Alice Lindsey|       PBCS|  Private Banking -...|PBCS-45|TRD0050002|
|          PB|2007-08-10 12:23:18|Callie Flowers|       PBCS|  Private Banking -...|PBCS-77|TRD0050004|
|          PB|2015-04-14 02:40:12|    Maryam Fry|       PBCS|  Private Banking -...|PBCS-66|TRD0050006|
|          PB|2009-01-10 09:40:54|   Gemma Boyle|       PBCS|  Private Banking -...|PBCS-65|TRD0050008|
|          PB|2012-10-13 23:05:53|Knox Cleveland|       PBCS|  Private Banking -...|PBCS-68|TRD0050010|
+------------+-------------------+--------------+-----------+----------------------+-------+----------+
only showing top 5 rows



## You can directly load into a Pandas Dataframe

In [5]:
client.get_pandas_dataframe(spark,data_set_name='stocks')

,Adj Close,Close,Date,High,Low,Open,Volume
0,118.250000,118.250000,2016-10-25,118.360001,117.309998,117.949997,39190300.0
1,116.599998,116.599998,2016-10-21,116.910004,116.279999,116.809998,23192700.0
2,117.120003,117.120003,2016-10-19,117.760002,113.800003,117.250000,20034600.0
3,117.550003,117.550003,2016-10-17,117.839996,116.779999,117.330002,23624900.0
4,116.980003,116.980003,2016-10-13,117.440002,115.720001,116.790001,35192400.0
...,...,...,...,...,...,...,...
248,119.033154,121.059998,2015-11-06,121.809998,120.620003,121.110001,33042300.0
249,119.446126,122.000000,2015-11-04,123.820000,121.620003,123.129997,44886100.0
250,118.643292,121.180000,2015-11-02,121.360001,119.610001,120.800003,32203300.0
251,118.006897,120.529999,2015-10-29,120.690002,118.269997,118.699997,51227300.0


In [12]:
import seaborn as sns
pandasDF = client.get_pandas_dataframe(spark,data_set_name='stocks')
cm = sns.light_palette("blue", as_cmap=True)
pandasDF[(pandasDF['Date']>'2015-11-19 00:00:00') & (pandasDF['Date']<'2015-12-07 00:00:00')].style.background_gradient(cmap=cm)


,Adj Close,Close,Date,High,Low,Open,Volume
112,113.271265,115.199997,2015-12-03 00:00:00,116.790001,114.220001,116.550003,41569500.000000
113,115.375435,117.339996,2015-12-01 00:00:00,118.809998,116.860001,118.750000,34852400.000000
114,115.837567,117.809998,2015-11-27 00:00:00,118.410004,117.599998,118.290001,13046400.000000
115,116.889653,118.879997,2015-11-24 00:00:00,119.349998,117.120003,117.330002,42803200.000000
116,117.302627,119.300003,2015-11-20 00:00:00,119.919998,118.849998,119.199997,34287100.000000
238,117.037143,119.029999,2015-12-04 00:00:00,119.250000,115.110001,115.290001,57777000.000000
239,114.333185,116.279999,2015-12-02 00:00:00,118.110001,116.080002,117.339996,33386600.000000
240,116.319369,118.300003,2015-11-30 00:00:00,119.410004,117.750000,117.989998,39180300.000000
241,116.053885,118.029999,2015-11-25 00:00:00,119.230003,117.919998,119.209999,21388300.000000
242,115.778574,117.750000,2015-11-23 00:00:00,119.730003,117.339996,119.269997,32482500.000000


## Execute a SQL Query 

In [6]:
df.createOrReplaceTempView("employees")
result = spark.sql('''
   SELECT businessUnit, COUNT(*) as total
   FROM employees
   GROUP BY businessUnit
   ''')
result.show()

+------------+-----+
|businessUnit|total|
+------------+-----+
|          RA|   26|
|          HR|   49|
|          PB|  199|
|          IN|  263|
|          CM|   18|
|          IT|  108|
|          GC|   31|
|          WM|  192|
|          CF|   74|
|          RM|   46|
|        null|    1|
+------------+-----+



## You can also use Koverse queries to access this data
## this returns an array of records


In [9]:
results = client.query({'orgCode': 'PBCS-69'}, datasets=['employees'])
pd.DataFrame(list(results[0].records))


,orgCategory,orgCategoryDescription,hireDate,businessUnit,orgCode,name,traderId
0,PBCS,Private Banking - Client Services,"(42294, 12, 16, 18, 0, 0, 6, 350, 0)",PB,PBCS-69,Cassady Spence,TRD0050001
1,PBCS,Private Banking - Client Services,"(44076, 1, 27, 4, 46, 40, 0, 27, 0)",PB,PBCS-69,Uma Olsen,TRD0050009
2,PBCS,Private Banking - Client Services,"(42252, 4, 18, 14, 20, 0, 6, 109, 0)",PB,PBCS-69,Jasmine Castaneda,TRD0050013
3,PBCS,Private Banking - Client Services,"(46052, 4, 29, 13, 10, 0, 0, 120, 0)",PB,PBCS-69,Garth David,null
4,PBCS,Private Banking - Client Services,"(39042, 2, 17, 8, 6, 40, 3, 48, 0)",PB,PBCS-69,Igor Juarez,TRD0050012
5,PBCS,Private Banking - Client Services,"(44890, 5, 6, 19, 33, 20, 5, 126, 0)",PB,PBCS-69,Kelsie Gamble,TRD0050036
6,PBCS,Private Banking - Client Services,"(43134, 3, 5, 19, 43, 20, 0, 64, 0)",PB,PBCS-69,Derek Mcintosh,TRD0050050
7,PBCS,Private Banking - Client Services,"(40793, 5, 29, 14, 6, 40, 5, 149, 0)",PB,PBCS-69,Ulysses Kelly,TRD0050068
8,PBCS,Private Banking - Client Services,"(42033, 7, 11, 5, 36, 40, 0, 192, 0)",PB,PBCS-69,Quinlan Sellers,TRD0050132


## You can also return right into a pandas dataframe

In [7]:
client.query_as_pandas_dataframe({'orgCode': 'PBCS-69'}, 'employees')

,orgCategory,orgCategoryDescription,hireDate,businessUnit,orgCode,name,traderId
0,PBCS,Private Banking - Client Services,"(39042, 2, 17, 8, 6, 40, 3, 48, 0)",PB,PBCS-69,Igor Juarez,TRD0050012
1,PBCS,Private Banking - Client Services,"(44890, 5, 6, 19, 33, 20, 5, 126, 0)",PB,PBCS-69,Kelsie Gamble,TRD0050036
2,PBCS,Private Banking - Client Services,"(43134, 3, 5, 19, 43, 20, 0, 64, 0)",PB,PBCS-69,Derek Mcintosh,TRD0050050
3,PBCS,Private Banking - Client Services,"(40793, 5, 29, 14, 6, 40, 5, 149, 0)",PB,PBCS-69,Ulysses Kelly,TRD0050068
4,PBCS,Private Banking - Client Services,"(42033, 7, 11, 5, 36, 40, 0, 192, 0)",PB,PBCS-69,Quinlan Sellers,TRD0050132
5,PBCS,Private Banking - Client Services,"(42294, 12, 16, 18, 0, 0, 6, 350, 0)",PB,PBCS-69,Cassady Spence,TRD0050001
6,PBCS,Private Banking - Client Services,"(44076, 1, 27, 4, 46, 40, 0, 27, 0)",PB,PBCS-69,Uma Olsen,TRD0050009
7,PBCS,Private Banking - Client Services,"(42252, 4, 18, 14, 20, 0, 6, 109, 0)",PB,PBCS-69,Jasmine Castaneda,TRD0050013
8,PBCS,Private Banking - Client Services,"(46052, 4, 29, 13, 10, 0, 0, 120, 0)",PB,PBCS-69,Garth David,null


## This is a query with one part of a Field

In [9]:
results = client.query({'orgCategoryDescription': 'Private'}, datasets=['employees'])
pd.DataFrame(list(results[0].records))

,orgCategory,orgCategoryDescription,hireDate,businessUnit,orgCode,name,traderId
0,PBCS,Private Banking - Client Services,"(42002, 12, 19, 13, 3, 20, 3, 353, 0)",PB,PBCS-45,Alice Lindsey,TRD0050002
1,PBCS,Private Banking - Client Services,"(39576, 7, 26, 4, 20, 0, 0, 208, 0)",PB,PBCS-77,Callie Flowers,TRD0050004
2,PBCS,Private Banking - Client Services,"(47252, 7, 19, 6, 0, 0, 4, 201, 0)",PB,PBCS-66,Maryam Fry,TRD0050006
3,PBCS,Private Banking - Client Services,"(40997, 3, 25, 9, 40, 0, 5, 84, 0)",PB,PBCS-65,Gemma Boyle,TRD0050008
4,PBCS,Private Banking - Client Services,"(44755, 3, 4, 10, 3, 20, 4, 63, 0)",PB,PBCS-68,Knox Cleveland,TRD0050010
...,...,...,...,...,...,...,...
200,PBBD,Private Banking - Business Development,"(38027, 11, 4, 1, 50, 0, 3, 308, 0)",PB,PBBD-44,Neville Hogan,null
201,PBPC,Private Banking - Process Management,"(45710, 11, 8, 18, 16, 40, 5, 312, 0)",PB,PBPC-15,Karleigh Wong,null
202,PBBD,Private Banking - Business Development,"(39078, 8, 3, 21, 53, 20, 5, 215, 0)",PB,PBBD-42,Ignatius Bernard,null
203,WMPF,Wealth Management - Private Funds,"(38905, 1, 2, 18, 50, 0, 4, 2, 0)",WM,WMPF-26,Jemima Richard,null


In [10]:
client.query_as_pandas_dataframe({'DESCRIPTION': 'Protein'}, 'Synthea Observations')


,DATE,CODE,ENCOUNTER,DESCRIPTION,PATIENT,VALUE,UNITS,TYPE
0,"(52221, 7, 19, 0, 0, 0, 3, 200, 0)",2885-2,2b1b3534-074e-43d0-90aa-cc85d3a5c3c5,Protein [Mass/volume] in Serum or Plasma,9aac209e-5ae3-4b3a-ae00-26a73c0e0a94,69.7,g/dL,numeric
1,"(33258, 10, 20, 0, 0, 0, 6, 293, 0)",2885-2,9d1d3067-03dc-4538-807d-afea103af47d,Protein [Mass/volume] in Serum or Plasma,fbb1366a-5205-4b34-888f-50dda8dbe03c,75.3,g/dL,numeric
2,"(37256, 2, 23, 0, 0, 0, 2, 54, 0)",2885-2,8478267d-88ca-4df1-8ebf-341565881b77,Protein [Mass/volume] in Serum or Plasma,fbb1366a-5205-4b34-888f-50dda8dbe03c,70.9,g/dL,numeric
3,"(38255, 6, 26, 0, 0, 0, 1, 177, 0)",2885-2,5630e814-63ca-4cc2-b27e-197a16c79af4,Protein [Mass/volume] in Serum or Plasma,fbb1366a-5205-4b34-888f-50dda8dbe03c,75.2,g/dL,numeric
4,"(39254, 10, 26, 0, 0, 0, 0, 299, 0)",2885-2,b2eab71b-5cea-4abf-b542-da24dd33c433,Protein [Mass/volume] in Serum or Plasma,fbb1366a-5205-4b34-888f-50dda8dbe03c,74.7,g/dL,numeric
...,...,...,...,...,...,...,...,...
9995,"(39898, 3, 24, 0, 0, 0, 3, 83, 0)",2885-2,2d6fe2bb-7c86-4c3a-bf58-dc94fd81adf8,Protein [Mass/volume] in Serum or Plasma,ead182ec-6dad-4c11-92b7-a07f601b054d,79.8,g/dL,numeric
9996,"(40897, 7, 24, 0, 0, 0, 2, 205, 0)",2885-2,ec5c0305-6173-4c4b-9358-bef3784756fd,Protein [Mass/volume] in Serum or Plasma,ead182ec-6dad-4c11-92b7-a07f601b054d,64.1,g/dL,numeric
9997,"(43895, 7, 28, 0, 0, 0, 6, 209, 0)",2885-2,558fae8f-9047-48bc-92a0-63ac215b4e15,Protein [Mass/volume] in Serum or Plasma,ead182ec-6dad-4c11-92b7-a07f601b054d,63.5,g/dL,numeric
9998,"(44894, 11, 27, 0, 0, 0, 5, 331, 0)",2885-2,dbef831a-f116-4a0f-99f8-3a6141d13697,Protein [Mass/volume] in Serum or Plasma,ead182ec-6dad-4c11-92b7-a07f601b054d,64.3,g/dL,numeric


In [5]:
#. A query with a range from 100 to 114
results = client.query({'Close': {'$gt': 100.0,'$lt': 114.0}}, datasets=['stocks'])
pd.DataFrame(list(results[0].records))

,High,Low,Volume,Adj Close,Close,Date,Open
0,113.660004,112.690002,21453100,113.050003,113.050003,"(48730, 9, 19, 0, 0, 0, 4, 262, 0)",113.400002
1,113.050003,112.279999,21701800,112.519997,112.519997,"(48725, 3, 29, 0, 0, 0, 6, 88, 0)",112.709999
2,113.800003,111.800003,35887000,112.180000,112.180000,"(48714, 4, 16, 0, 0, 0, 3, 106, 0)",113.160004
3,113.180000,112.339996,24607400,113.089996,113.089996,"(48708, 10, 24, 0, 0, 0, 5, 298, 0)",113.000000
4,114.790001,111.550003,52481200,112.709999,112.709999,"(48697, 11, 10, 0, 0, 0, 2, 314, 0)",114.419998
...,...,...,...,...,...,...,...
105,108.849998,107.199997,32657400,106.791601,108.610001,"(47944, 12, 8, 0, 0, 0, 4, 343, 0)",107.269997
106,107.370003,105.570000,47590600,105.533033,107.330002,"(47939, 6, 18, 0, 0, 0, 6, 169, 0)",107.279999
107,112.250000,108.980003,44772800,107.155409,108.980003,"(47928, 7, 5, 0, 0, 0, 3, 187, 0)",112.019997
108,112.800003,110.349998,52978100,108.640123,110.489998,"(47923, 1, 13, 0, 0, 0, 5, 13, 0)",111.940002


# Get Samples

In [31]:
samples = client.getSamples(dataset = 'employees', maxRecords=100, removeByteArrays=False, maxStringLength=100)
pprint.pprint(list(samples)[0:4])


[{'businessUnit': 'IN',
  'hireDate': time.struct_time(tm_year=40242, tm_mon=4, tm_mday=22, tm_hour=20, tm_min=3, tm_sec=20, tm_wday=4, tm_yday=112, tm_isdst=0),
  'name': 'Charde Moore',
  'orgCategory': 'INTR',
  'orgCategoryDescription': 'Investments - Trading',
  'orgCode': 'INTR-345',
  'traderId': 'TRD0050376'},
 {'businessUnit': 'PB',
  'hireDate': time.struct_time(tm_year=40008, tm_mon=8, tm_mday=22, tm_hour=16, tm_min=20, tm_sec=0, tm_wday=4, tm_yday=235, tm_isdst=0),
  'name': 'Evelyn Carr',
  'orgCategory': 'PBCS',
  'orgCategoryDescription': 'Private Banking - Client Services',
  'orgCode': 'PBCS-67',
  'traderId': 'TRD0050130'},
 {'businessUnit': 'IT',
  'hireDate': time.struct_time(tm_year=41747, tm_mon=10, tm_mday=24, tm_hour=1, tm_min=43, tm_sec=20, tm_wday=1, tm_yday=297, tm_isdst=0),
  'name': 'Cheyenne Sandoval',
  'orgCategory': 'ITIS',
  'orgCategoryDescription': 'Information Technology - Integration Services',
  'orgCode': 'ITIS-75',
  'traderId': 'null'},
 {'busi

## Save back to Koverse

### Load dataframe and create new column

In [11]:
from koverse.client import DataSetConfiguration
from pyspark.sql.functions import lit
df = client.get_spark_dataframe(spark,data_set_name='employees')
df = df.withColumn("newColumn",lit('NewValue'))
df.show(4)


+------------+-------------------+--------------+-----------+----------------------+-------+----------+---------+
|businessUnit|           hireDate|          name|orgCategory|orgCategoryDescription|orgCode|  traderId|newColumn|
+------------+-------------------+--------------+-----------+----------------------+-------+----------+---------+
|          PB|2010-04-29 09:19:48|Cassady Spence|       PBCS|  Private Banking -...|PBCS-69|TRD0050001| NewValue|
|          PB|2013-06-27 22:23:36|  Ashely Wyatt|       PBCS|  Private Banking -...|PBCS-79|TRD0050003| NewValue|
|          PB|2007-01-22 15:37:49|   Thor Harper|       PBCS|  Private Banking -...|PBCS-65|TRD0050005| NewValue|
|          PB|2009-03-02 12:09:39|  Otto Baldwin|       PBCS|  Private Banking -...|PBCS-65|TRD0050007| NewValue|
+------------+-------------------+--------------+-----------+----------------------+-------+----------+---------+
only showing top 4 rows



### Save to Koverse

In [12]:
client.write_spark_dataframe(spark, df, DataSetConfiguration("employees_new", "overwrite", False))

In [2]:
client.get_pandas_dataframe(spark,data_set_name='employees_new')

,businessUnit,hireDate,name,newColumn,orgCategory,orgCategoryDescription,orgCode,traderId
0,PB,2010-04-29 09:19:48,Cassady Spence,NewValue,PBCS,Private Banking - Client Services,PBCS-69,TRD0050001
1,PB,2013-06-27 22:23:36,Ashely Wyatt,NewValue,PBCS,Private Banking - Client Services,PBCS-79,TRD0050003
2,PB,2007-01-22 15:37:49,Thor Harper,NewValue,PBCS,Private Banking - Client Services,PBCS-65,TRD0050005
3,PB,2009-03-02 12:09:39,Otto Baldwin,NewValue,PBCS,Private Banking - Client Services,PBCS-65,TRD0050007
4,PB,2012-02-08 22:15:10,Uma Olsen,NewValue,PBCS,Private Banking - Client Services,PBCS-69,TRD0050009
...,...,...,...,...,...,...,...,...
1002,WM,2010-01-23 06:28:57,Rahim Rose,NewValue,WMCP,Wealth Management - Compliance,WMCP-11,null
1003,IT,2009-09-08 01:12:32,Marah Fitzpatrick,NewValue,ITSP,Information Technology - Special Projects,ITSP-90,null
1004,RM,2005-12-09 23:36:57,Jeff Davis,NewValue,RMPM,Risk Management - Portfolio Management,RMPM-19,null
1005,IT,2006-11-23 00:33:00,Jodean Bell,NewValue,ITAM,Information Technology - Application Management,ITAM-54,null
